In [48]:
import pandas as pd
from simple_salesforce import Salesforce as sf
from simple_salesforce import bulk
import teradatasql


login_sf = sf(username = 'intl_sales_ops_analysts@groupon.com', password = 'BA_intl_grp_3', security_token = 'ryDPBLSTkb54qbEY06GJSiA0')
con = teradatasql.connect(host='tdwd.group.on',user='ub_intl_sales_ops_2',password='BA_intl_grp_4')

countries = ['ES','FR','IT','DE','NL','BE','PL','AE','AU','GB','IE']  

for country in countries:


    ### This loads in the Teradata dataset
    td_data = pd.read_sql(f"SELECT account_id,pds_boost_flag,required_service,load_date FROM sandbox.pds_boost_account_flags WHERE load_date = (SELECT Max(load_date) FROM sandbox.pds_boost_account_flags) AND CASE when feature_country = 'UK' then 'GB' else feature_country end = '{country}'",con)
    td_data.columns = ['td_id','td_PDS_Boost__c','td_PDS_Boost_Summary__c','td_PDS_Boost_Date__c']
    
    td_data['td_PDS_Boost__c'] = td_data['td_PDS_Boost__c'].replace([1,0],['True','False'])
    td_data['td_PDS_Boost_Date__c'] = td_data['td_PDS_Boost_Date__c'].astype(str)
    td_data['td_PDS_Boost_Summary__c'] = td_data['td_PDS_Boost_Summary__c'].str[:255]
    
    ### This loads in the Salesforce dataset
    soql_output = login_sf.bulk.Account.query(f"SELECT Account_ID_18__c,PDS_Boost_Summary__c,PDS_Boost_Date__c,PDS_Boost__c FROM Account WHERE Feature_Country__c = '{country}' AND PDS_Boost_Summary__c != Null limit 200000")


    dic = {}
    dic['sf_id'] = [ x['Account_ID_18__c'] for x in soql_output ]
    dic['sf_PDS_Boost_Summary__c'] = [ x['PDS_Boost_Summary__c'] for x in soql_output ]
    dic['sf_PDS_Boost_Date__c'] = [ x['PDS_Boost_Date__c'] for x in soql_output ]
    dic['sf_PDS_Boost__c'] = [ x['PDS_Boost__c'] for x in soql_output ]
        
    sf_data = pd.DataFrame(dic, columns=dic.keys())
    sf_data['sf_PDS_Boost_Date__c'] = sf_data['sf_PDS_Boost_Date__c'].astype(str)
    sf_data['sf_PDS_Boost_Summary__c'] = sf_data['sf_PDS_Boost_Summary__c'].str[:255]
    

    ### Built two different datasets, one for changes to be uploaded (df) and one for values to be deleted (df_del)
    df = td_data.merge(sf_data, left_on='td_id', right_on='sf_id', how='left')
    df_del = sf_data.merge(td_data, left_on='sf_id', right_on='td_id', how='left')

    ### Filter the datasets built so we limit the changes to be made
    df2 = df[((df['sf_PDS_Boost_Summary__c'] == df['td_PDS_Boost_Summary__c']) & (df['sf_PDS_Boost_Date__c'].astype(str) != df['td_PDS_Boost_Date__c'].astype(str)))] # data where only a date change is needed
    df3 = df[(df['sf_PDS_Boost_Summary__c'] != df['td_PDS_Boost_Summary__c'])] # data where a full change is needed
    df_del = df_del[(df_del['td_id'] == None)]


    ### Data prep the deletion data
    # this drops unnecesary columns
    df_del.drop('td_id', axis=1, inplace=True)
    df_del.drop('td_PDS_Boost__c', axis=1, inplace=True)
    df_del.drop('td_PDS_Boost_Date__c', axis=1, inplace=True)
    df_del.drop('td_PDS_Boost_Summary__c', axis=1, inplace=True)

    # this adjusts the values in order to be ready for upload
    df_del.columns = ['id','PDS_Boost_Summary__c','PDS_Boost_Date__c','PDS_Boost__c']
    df_del['PDS_Boost__c'] = 'false'
    df_del['PDS_Boost_Summary__c'] = None
    df_del['PDS_Boost_Date__c'] = None

    if not df_del.empty:
        # prep data for upload and upload to SF
        ready_for_upload = []

        for row in df_del.itertuples():
            d = row._asdict()
            del d['Index']
            ready_for_upload.append(d)
            
        login_sf.bulk.Account.update(ready_for_upload,batch_size=10000,use_serial=True)
        
    else:
        pass


    ### Data prep the data where only a date change is needed
    # this drops unnecesary columns
    df2.drop('sf_id', axis=1, inplace=True)
    df2.drop('td_PDS_Boost__c', axis=1, inplace=True)
    df2.drop('sf_PDS_Boost__c', axis=1, inplace=True)
    df2.drop('sf_PDS_Boost_Summary__c', axis=1, inplace=True)
    df2.drop('td_PDS_Boost_Summary__c', axis=1, inplace=True)
    df2.drop('sf_PDS_Boost_Date__c', axis=1, inplace=True)

    # columns are renamed to fit Salesforce names
    df2.columns = ['Account_ID_18__c','PDS_Boost_Date__c']

    if not df2.empty:
        # prep data for upload and upload to SF
        ready_for_upload = []

        for row in df2.itertuples():
            d = row._asdict()
            del d['Index']
            ready_for_upload.append(d)

        login_sf.bulk.Account.update(ready_for_upload,batch_size=10000,use_serial=True)
        
    else:
        pass

    ### Data prep the data where a full change is needed
    # this drops unnecesary columns    
    df3.drop('sf_id', axis=1, inplace=True)
    df3.drop('sf_PDS_Boost__c', axis=1, inplace=True)
    df3.drop('sf_PDS_Boost_Date__c', axis=1, inplace=True)
    df3.drop('sf_PDS_Boost_Summary__c', axis=1, inplace=True)

    df3.columns = ['id','PDS_Boost__c','PDS_Boost_Summary__c','PDS_Boost_Date__c']

    df3['PDS_Boost_Date__c'] = df3['PDS_Boost_Date__c'].astype(str)
    df3['PDS_Boost__c'] = df3['PDS_Boost__c'].astype(bool)

    if not df3.empty:
        # prep data for upload and upload to SF
        ready_for_upload = []

        for row in df3.itertuples():
            d = row._asdict()
            del d['Index']
            ready_for_upload.append(d)

        login_sf.bulk.Account.update(ready_for_upload,batch_size=10000,use_serial=True)
        
    else:
        pass
    

    print(str(len(df3))+' full changes made in SF for '+country)
    print(str(len(df2))+' date changes made in SF for '+country)
    print(str(len(df_del))+' values deleted from SF for '+country)
    print('Process completed for '+country)

print('Process completed')


c:\Users\markd\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
